# Reference 

1. open dart official document
https://dart-fss.readthedocs.io/en/latest/welcome.html

2. 코딩 유치원 
https://coding-kindergarten.tistory.com/85

In [1]:
# install
!pip install dart-fss

In [1]:
# xml 파일을 읽기 위한 라이브러리
# from xml.etree.ElementTree import parse
# 판다스 
import pandas as pd
import os
import dart_fss as dart

My api key :7db60698e63bd621a006025500fe0436ee65cb47

dart 서비스에 올라가 있는 기업들은 비상장 기업들의 재무정보도 올라와 있다. 
우리는 상장사의 재무정보만 보고 싶은 것이기 때문에, 전체 corp_list 중에서 상장기업들만 추출해야 한다.   

상장사 리스트를 얻기 위해 한국거래소 > 기본 통계 > 주식 > 종목정보 > 전종목 기본정보에서 전종목 기본정보를 다운받는다.   
http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201040201

In [2]:
# 다운받은 전종목 기본정보 불러오기 
all_stocks = pd.read_excel('./data_5655_20220311.xlsx')

C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
all_stocks.head(1)

,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수
0,KR7098120009,098120,(주)마이크로컨텍솔루션,마이크로컨텍솔,"Micro Contact Solution Co.,Ltd.",2008/09/23,KOSDAQ,주권,벤처기업부,보통주,500,8312766


dart 패키지에서는 주식종목코드를 사용하는 것이아니라, dart 자체적인 corp_code(기업코드)를 가지고 정보를 불러온다.  
따라서 한국거래소에서 얻은 상장기업 주식코드를 가지고, dart의 데이터에서 상장기업 리스트와, 그들의 corp_code를 추출한다. 
여기서 주의 할 것은, 한국거래소에서 얻은 상장기업 주식코드에는 우선주도 포함된다는 점이다. 상장기업 주식코드중 보통주만 사용해야한다. 

In [4]:
# 상장기업 주식 코드 중 보통주만 걸러내기 
listed_companies = all_stocks[all_stocks['주식종류']=='보통주']

In [5]:
import dart_fss as dart

# Open DART API KEY 설정
api_key='7db60698e63bd621a006025500fe0436ee65cb47'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.corp.get_corp_list()

Output()

Output()

Output()

In [6]:
# 상장 기업들의 기업코드와 회사 이름
listed_companies_corp_code = []
listed_companies_name = []

for stock_code in listed_companies['단축코드'] : 
    #corp_list.find_by_corp_name( ) 함수의 결과 데이터는 [[종목 코드]종목 이름]의 형태
    #이를 벗겨내기 위해서 corp_list.find_by_corp_name( )[0]으로 코딩함 
    try : 
        listed_companies_corp_code.append(corp_list.find_by_stock_code(stock_code).corp_code)
        listed_companies_name.append(corp_list.find_by_stock_code(stock_code).corp_name)
        
    except : 
        print(stock_code)
        

225330


In [7]:
# 에러 확인. 현재 무슨일인지 모르겠지만 주식코드 225330, 씨엠에스에듀주식회사 정보를 얻어올 수 없다
corp_list.find_by_stock_code('225330')

(우선주를 제외한)상장기업들 수

In [8]:
len(listed_companies_name)

2498

In [9]:
corp_code_df = pd.DataFrame({'기업이름':listed_companies_name, '기업코드' : listed_companies_corp_code})
corp_code_df.head(5)

,기업이름,기업코드
0,마이크로컨텍솔,00568188
1,스카이이앤엠,00814786
2,포스코엠텍,00128111
3,AJ네트웍스,00365387
4,AK홀딩스,00125080


In [10]:
# 삼성전자 code
corp_code = '00126380'

# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()

# 삼성전자
samsung = corp_list.find_by_corp_code(corp_code=corp_code)

# 2019년 01월 01일 부터 연결재무제표 검색
# fs = samsung.extract_fs(bgn_de='20120101') 와 동일
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20190101')

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

## 재무상태표(balance sheet(bs) or financial statement sheet) 가공

In [11]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
bs_df = fs.show('bs', show_class = False, show_concept = False).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
bs_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
# 영어이름이 혹시 필요하다면, 바로 아래 코드만 주석처리 해주면 된다. 지금은 일단 제거
bs_df.drop(1, inplace = True)

In [12]:
bs_df.columns = bs_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
bs_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.

In [13]:
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
bs_df.rename(columns={bs_df.columns[0]:'Account_year'}, inplace = True) 
bs_df

,Account_year,유동자산,현금및현금성자산,단기금융상품,단기상각후원가금융자산,단기당기손익-공정가치금융자산,매출채권,미수금,선급비용,재고자산,...,자본금,우선주자본금,보통주자본금,주식발행초과금,이익잉여금(결손금),기타자본항목,매각예정분류기타자본항목,비지배지분,자본총계,부채와자본총계
2,20211231,"218,163,185,000,000.0","39,031,415,000,000.0","81,708,986,000,000.0","3,369,034,000,000.0","40,757,000,000.0","40,713,415,000,000.0","4,497,257,000,000.0","2,336,252,000,000.0","41,384,404,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","293,064,763,000,000.0","-2,128,473,000,000.0",0.0,"8,662,234,000,000.0","304,899,931,000,000.0","426,621,158,000,000.0"
3,20201231,"198,215,579,000,000.0","29,382,578,000,000.0","92,441,703,000,000.0","2,757,111,000,000.0","71,451,000,000.0","30,965,058,000,000.0","3,604,539,000,000.0","2,266,100,000,000.0","32,043,145,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","271,068,211,000,000.0","-8,687,155,000,000.0","-12,132,000,000.0","8,277,685,000,000.0","275,948,016,000,000.0","378,235,718,000,000.0"
4,20191231,"181,385,260,000,000.0","26,885,999,000,000.0","76,252,052,000,000.0","3,914,216,000,000.0","1,727,436,000,000.0","35,131,343,000,000.0","4,179,120,000,000.0","2,406,220,000,000.0","26,766,464,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","254,582,894,000,000.0","-4,968,829,000,000.0",0.0,"7,964,949,000,000.0","262,880,421,000,000.0","352,564,497,000,000.0"
5,20181231,"174,697,424,000,000.0","30,340,505,000,000.0","65,893,797,000,000.0","2,703,693,000,000.0","2,001,948,000,000.0","33,867,733,000,000.0","3,080,733,000,000.0","4,136,167,000,000.0","28,984,704,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","242,698,956,000,000.0","-7,931,370,000,000.0",0.0,"7,684,184,000,000.0","247,753,177,000,000.0","339,357,244,000,000.0"
6,20171231,"146,982,464,000,000.0","30,545,130,000,000.0","49,447,696,000,000.0",NaN,NaN,"27,695,995,000,000.0","4,108,961,000,000.0","3,835,219,000,000.0","24,983,355,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","215,811,200,000,000.0","-13,899,191,000,000.0",NaN,"7,278,012,000,000.0","214,491,428,000,000.0","301,752,090,000,000.0"
7,20161231,"141,429,704,000,000.0","32,111,442,000,000.0","52,432,411,000,000.0",NaN,NaN,"24,279,211,000,000.0","3,521,197,000,000.0","3,502,083,000,000.0","18,353,503,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","193,086,317,000,000.0","-11,934,586,000,000.0","-28,810,000,000.0","6,538,705,000,000.0","192,963,033,000,000.0","262,174,324,000,000.0"


## 손익계산서 가공
'ci'는 포괄 손익 계산서, 
'is'는 손익계산서.   
둘 중 무엇으로?

In [14]:
is_df = fs.show('is', show_class = False, show_concept = True).T.reset_index()

In [15]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
is_df = fs.show('is', show_class = False, show_concept = False).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
is_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
# 영어이름이 혹시 필요하다면, 바로 아래 코드만 주석처리 해주면 된다. 지금은 일단 제거
is_df.drop(1, inplace = True)

In [16]:
is_df.columns = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.

In [17]:
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
is_df.rename(columns={is_df.columns[0]:'Account_year'}, inplace = True) 
is_df

,Account_year,수익(매출액),매출원가,매출총이익,판매비와관리비,영업이익,기타수익,기타비용,지분법이익,금융수익,금융비용,법인세비용차감전순이익(손실),법인세비용,계속영업이익(손실),당기순이익(손실),지배기업의 소유주에게 귀속되는 당기순이익(손실),비지배지분에 귀속되는 당기순이익(손실),기본주당이익(손실),희석주당이익(손실)
2,20210101-20211231,"279,604,799,000,000.0","166,411,342,000,000.0","113,193,457,000,000.0","61,559,601,000,000.0","51,633,856,000,000.0","2,205,695,000,000.0","2,055,971,000,000.0","729,614,000,000.0","8,543,187,000,000.0","7,704,554,000,000.0","53,351,827,000,000.0","13,444,377,000,000.0","39,907,450,000,000.0","39,907,450,000,000.0","39,243,791,000,000.0","663,659,000,000.0","5,777.0","5,777.0"
3,20200101-20201231,"236,806,988,000,000.0","144,488,296,000,000.0","92,318,692,000,000.0","56,324,816,000,000.0","35,993,876,000,000.0","1,384,068,000,000.0","2,488,902,000,000.0","506,530,000,000.0","12,267,600,000,000.0","11,318,055,000,000.0","36,345,117,000,000.0","9,937,285,000,000.0","26,407,832,000,000.0","26,407,832,000,000.0","26,090,846,000,000.0","316,986,000,000.0","3,841.0","3,841.0"
4,20190101-20191231,"230,400,881,000,000.0","147,239,549,000,000.0","83,161,332,000,000.0","55,392,823,000,000.0","27,768,509,000,000.0","1,778,666,000,000.0","1,414,707,000,000.0","412,960,000,000.0","10,161,632,000,000.0","8,274,871,000,000.0","30,432,189,000,000.0","8,693,324,000,000.0","21,738,865,000,000.0","21,738,865,000,000.0","21,505,054,000,000.0","233,811,000,000.0","3,166.0","3,166.0"
5,20180101-20181231,"243,771,415,000,000.0","132,394,411,000,000.0","111,377,004,000,000.0","52,490,335,000,000.0","58,886,669,000,000.0","1,485,037,000,000.0","1,142,018,000,000.0","539,845,000,000.0","9,999,321,000,000.0","8,608,896,000,000.0","61,159,958,000,000.0","16,815,101,000,000.0","44,344,857,000,000.0","44,344,857,000,000.0","43,890,877,000,000.0","453,980,000,000.0","6,461.0","6,461.0"
6,20170101-20171231,"239,575,376,000,000.0","129,290,661,000,000.0","110,284,715,000,000.0","56,639,677,000,000.0","53,645,038,000,000.0","3,010,657,000,000.0","1,419,648,000,000.0","201,442,000,000.0","9,737,391,000,000.0","8,978,913,000,000.0","56,195,967,000,000.0","14,009,220,000,000.0","42,186,747,000,000.0","42,186,747,000,000.0","41,344,569,000,000.0","842,178,000,000.0","5,997.0","5,997.0"
7,20160101-20161231,"201,866,745,000,000.0","120,277,715,000,000.0","81,589,030,000,000.0","52,348,358,000,000.0","29,240,672,000,000.0","3,238,261,000,000.0","2,463,814,000,000.0","19,501,000,000.0","11,385,645,000,000.0","10,706,613,000,000.0","30,713,652,000,000.0","7,987,560,000,000.0","22,726,092,000,000.0","22,726,092,000,000.0","22,415,655,000,000.0","310,437,000,000.0","3,159.0","3,159.0"


## 현금흐름표 가공

In [18]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
cf_df = fs.show('cf', show_class = False, show_concept = False).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
cf_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
# 영어이름이 혹시 필요하다면, 바로 아래 코드만 주석처리 해주면 된다. 지금은 일단 제거
cf_df.drop(1, inplace = True)

In [19]:
cf_df.columns = cf_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
cf_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.

In [20]:
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
cf_df.rename(columns={cf_df.columns[0]:'Account_year'}, inplace = True) 
cf_df

,Account_year,영업활동 현금흐름,영업에서 창출된 현금흐름,당기순이익,조정,영업활동으로 인한 자산부채의 변동,이자의 수취,이자의 지급,배당금 수입,법인세 납부액,...,단기차입금의 순증가(감소),장기차입금의 차입,사채 및 장기차입금의 상환,배당금의 지급,비지배지분의 증감,매각예정분류,외화환산으로 인한 현금의 변동,현금및현금성자산의 순증감,기초의 현금및현금성자산,기말의 현금및현금성자산
2,20210101-20211231,"65,105,448,000,000.0","72,676,199,000,000.0","39,907,450,000,000.0","49,055,633,000,000.0","-16,286,884,000,000.0","1,406,706,000,000.0","434,441,000,000.0","299,033,000,000.0","8,842,049,000,000.0",...,"-2,616,943,000,000.0","58,279,000,000.0","894,749,000,000.0","20,510,350,000,000.0","-27,270,000,000.0","139,000,000.0","1,582,046,000,000.0","9,648,837,000,000.0","29,382,578,000,000.0","39,031,415,000,000.0"
3,20200101-20201231,"65,287,009,000,000.0","68,148,810,000,000.0","26,407,832,000,000.0","41,618,554,000,000.0","122,424,000,000.0","2,220,209,000,000.0","555,321,000,000.0","243,666,000,000.0","4,770,355,000,000.0",...,"2,191,186,000,000.0","14,495,000,000.0","864,947,000,000.0","9,676,760,000,000.0","8,187,000,000.0","-139,000,000.0","-833,861,000,000.0","2,496,579,000,000.0","26,885,999,000,000.0","29,382,578,000,000.0"
4,20190101-20191231,"45,382,915,000,000.0","56,635,791,000,000.0","21,738,865,000,000.0","37,442,682,000,000.0","-2,545,756,000,000.0","2,306,401,000,000.0","579,979,000,000.0","241,801,000,000.0","13,221,099,000,000.0",...,"865,792,000,000.0",0.0,"709,400,000,000.0","9,639,202,000,000.0","-1,700,000,000.0",0.0,"595,260,000,000.0","-3,454,506,000,000.0","30,340,505,000,000.0","26,885,999,000,000.0"
5,20180101-20181231,"67,031,863,000,000.0","78,025,064,000,000.0","44,344,857,000,000.0","43,604,573,000,000.0","-9,924,366,000,000.0","1,788,520,000,000.0","548,272,000,000.0","215,992,000,000.0","12,449,441,000,000.0",...,"-2,046,470,000,000.0","3,580,000,000.0","1,986,597,000,000.0","10,193,695,000,000.0","8,071,000,000.0",0.0,"94,187,000,000.0","-204,625,000,000.0","30,545,130,000,000.0","30,340,505,000,000.0"
6,20170101-20171231,"62,162,041,000,000.0","67,777,432,000,000.0","42,186,747,000,000.0","36,211,232,000,000.0","-10,620,547,000,000.0","1,581,117,000,000.0","542,715,000,000.0","173,305,000,000.0","6,827,098,000,000.0",...,"2,730,676,000,000.0","998,311,000,000.0","1,140,803,000,000.0","6,804,297,000,000.0","5,670,000,000.0",NaN,"-1,782,270,000,000.0","-1,566,312,000,000.0","32,111,442,000,000.0","30,545,130,000,000.0"
7,20160101-20161231,"47,385,644,000,000.0","52,299,610,000,000.0","22,726,092,000,000.0","30,754,471,000,000.0","-1,180,953,000,000.0","1,405,085,000,000.0","443,838,000,000.0","256,851,000,000.0","6,132,064,000,000.0",...,"1,351,037,000,000.0","1,041,743,000,000.0","252,846,000,000.0","3,114,742,000,000.0","13,232,000,000.0",NaN,"417,243,000,000.0","9,474,698,000,000.0","22,636,744,000,000.0","32,111,442,000,000.0"


## Integration 
재무상태표, 현금흐름표, 손익계산서 하나의 dataframe으로 통합.

In [21]:
integrated_df = pd.concat([bs_df,is_df,cf_df], axis=1)

In [28]:
integrated_df['지배기업 소유주지분']

2   296,237,697,000,000.0
3   267,670,331,000,000.0
4   254,915,472,000,000.0
5   240,068,993,000,000.0
6   207,213,416,000,000.0
7   186,424,328,000,000.0
Name: 지배기업 소유주지분, dtype: object

In [29]:
len(integrated_df.columns)

115

In [30]:
for i in integrated_df.columns : 
    print(i)

Account_year
유동자산
현금및현금성자산
단기금융상품
단기상각후원가금융자산
단기당기손익-공정가치금융자산
매출채권
미수금
선급비용
재고자산
기타유동자산
매각예정분류자산
비유동자산
기타포괄손익-공정가치금융자산
당기손익-공정가치금융자산
관계기업 및 공동기업 투자
유형자산
무형자산
순확정급여자산
이연법인세자산
기타비유동자산
자산총계
유동부채
매입채무
단기차입금
미지급금
선수금
예수금
미지급비용
당기법인세부채
유동성장기부채
충당부채
기타유동부채
매각예정분류부채
비유동부채
사채
장기차입금
장기미지급금
순확정급여부채
이연법인세부채
장기충당부채
기타비유동부채
부채총계
지배기업 소유주지분
자본금
우선주자본금
보통주자본금
주식발행초과금
이익잉여금(결손금)
기타자본항목
매각예정분류기타자본항목
비지배지분
자본총계
부채와자본총계
Account_year
수익(매출액)
매출원가
매출총이익
판매비와관리비
영업이익
기타수익
기타비용
지분법이익
금융수익
금융비용
법인세비용차감전순이익(손실)
법인세비용
계속영업이익(손실)
당기순이익(손실)
지배기업의 소유주에게 귀속되는 당기순이익(손실)
비지배지분에 귀속되는 당기순이익(손실)
기본주당이익(손실)
희석주당이익(손실)
Account_year
영업활동 현금흐름
영업에서 창출된 현금흐름
당기순이익
조정
영업활동으로 인한 자산부채의 변동
이자의 수취
이자의 지급
배당금 수입
법인세 납부액
투자활동 현금흐름
단기금융상품의 순감소(증가)
단기상각후원가금융자산의 순감소(증가)
단기당기손익-공정가치금융자산의 순감소(증가)
장기금융상품의 처분
장기금융상품의 취득
상각후원가금융자산의 처분
상각후원가금융자산의 취득
기타포괄손익-공정가치금융자산의 처분
기타포괄손익-공정가치금융자산의 취득
당기손익-공정가치금융자산의 처분
당기손익-공정가치금융자산의 취득
관계기업 및 공동기업 투자의 처분
관계기업 및 공동기업 투자의 취득
유형자산의 처분
유형자산의 취득
무형자산의 처분
무형자산의 취득
사업결합으로 인한 현금유출액
매각예정자산의 처분으로 인한 현금유입액
기타투자활동으

In [31]:
pd.DataFrame({'데이터정의서':['유동자산금액', '비유동자산금액', '자산총계금액', '유동부채금액', '비유동부채금액', '부채총계금액','소유주지분금액',
                      '비지배지분금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','매출총이익금액','판매비와관리비금액',
                      '영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','관계기업투자주식지분법손익금액',
                      '지분법이익금액','지분법손실금액','관계기업투자처분손익금액',
                       '관계기업투자손상차손금액','관계기업투자손상차손환입금액','기타영업외손익금액', '법인세비용금액','당기순이익금액',
                       '주당이익금액','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','외화환산현금변동금액',
                       '현금자산순증감금액','기초현금자산금액','기말현금자산금액'],
             '재무상태표상용어' : ['유동자산', '비유동자산', '자산총계','유동부채','비유동부채','부채총계', '지배기업 소유주지분','비지배지분','자본총계','부채와자본총계'
                                              ,'수익(매출액)','매출원가','매출총이익','판매비와관리비','계속영업이익(손실)??','금융수입','금융비용',
                           '기타수익?','기타비용?', '-' ,'지분법이익','-' , '관계기업 및 공동기업 투자의 처분??','-' ,'-' , '??','법인세 납부액','당기순이익',
                           '희석주당이익,기본주당이익(중 무엇)?', '영업활동 현금흐름','투자활동 현금흐름','재무활동 현금흐름','외화환산으로 인한 현금의변동',
                           '현금및현금성자산의 순증감','기초의 현금및현금성자산','기말의 현금및현금성자산']})

,데이터정의서,재무상태표상용어
0,유동자산금액,유동자산
1,비유동자산금액,비유동자산
2,자산총계금액,자산총계
3,유동부채금액,유동부채
4,비유동부채금액,비유동부채
5,부채총계금액,부채총계
6,소유주지분금액,지배기업 소유주지분
7,비지배지분금액,비지배지분
8,자본총계금액,자본총계
9,부채자본총계금액,부채와자본총계


## 문제점발견 :
1. 데이터 정의서 상의 용어와 재무제표상 용어가 다르다... 혹시 재무상태표상 용어가 기업마다도 다를까? 
2. 재무제표상 용어가 기업마다 약간씩 다를 수 있다. 

현대의 경우를 살펴보자.

In [32]:
corp_code_df[corp_code_df['기업이름']=='현대자동차']

,기업이름,기업코드
2428,현대자동차,00164742


In [33]:
# 현대자동차 code
corp_code = '00164742'

# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()

# 현대차
hyundai = corp_list.find_by_corp_code(corp_code=corp_code)

# 2019년 01월 01일 부터 연결재무제표 검색
# fs = samsung.extract_fs(bgn_de='20120101') 와 동일
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20190101')

Annual reports:   0%|          | 0/5 [00:00<?, ?report/s]

In [34]:
hyundai

Label,Data
corp_code,00164742
corp_name,현대자동차
stock_code,005380
modify_date,20220105
sector,자동차용 엔진 및 자동차 제조업
product,"자동차(승용차,버스,트럭,특장차),자동차부품,자동차전착도료 제조,차량정비사업"
corp_cls,Y


In [35]:
hy_bs_df = fs.show('bs', show_class = True, show_concept = True).T.reset_index()
hy_bs_df

,level_0,level_1,0,1,2,3,4,5,6,7,...,48,49,50,51,52,53,54,55,56,57
0,"[D210000] Statement of financial position, cur...",concept_id,ifrs-full_CurrentAssets,ifrs-full_CashAndCashEquivalents,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,ifrs-full_OtherCurrentFinancialAssets,dart_ShortTermTradeReceivable,ifrs-full_TradeAndOtherCurrentReceivables,ifrs-full_Inventories,dart_OtherCurrentAssets,...,ifrs-full_Liabilities,ifrs-full_EquityAttributableToOwnersOfParent,ifrs-full_IssuedCapital,dart_CapitalSurplus,dart_ElementsOfOtherStockholdersEquity,dart_OtherComprehensiveIncomeLossAccumulatedAm...,ifrs-full_RetainedEarnings,ifrs-full_NoncontrollingInterests,ifrs-full_Equity,ifrs-full_EquityAndLiabilities
1,"[D210000] Statement of financial position, cur...",label_ko,유동자산,현금및현금성자산,단기금융상품,기타금융자산,매출채권,기타채권,재고자산,기타자산,...,부채총계,지배기업소유주지분,자본금,자본잉여금,기타자본항목,기타포괄손익누계액,이익잉여금,비지배지분,자본총계,부채와자본총계
2,"[D210000] Statement of financial position, cur...",label_en,Current assets,Cash and cash equivalents,Short-term financial instruments,Other financial assets,Trade notes and accounts receivable,Other receivables,Inventories,Other assets,...,Total liabilities,Equity attributable to owners of parent,Capital stock,Capital surplus,Other capital items,Accumulated other comprehensive loss,Retained earnings,Non-controlling interests,Total equity,Total equity and liabilities
3,"[D210000] Statement of financial position, cur...",class0,재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],...,재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract],재무상태표 [abstract]
4,"[D210000] Statement of financial position, cur...",class1,자산 [abstract],자산 [abstract],자산 [abstract],자산 [abstract],자산 [abstract],자산 [abstract],자산 [abstract],자산 [abstract],...,부채 [abstract],자본 [abstract],자본 [abstract],자본 [abstract],자본 [abstract],자본 [abstract],자본 [abstract],자본 [abstract],자본 [abstract],부채와자본총계
5,"[D210000] Statement of financial position, cur...",class2,유동자산,유동자산,유동자산,유동자산,유동자산,유동자산,유동자산,유동자산,...,부채총계,지배기업소유주지분,지배기업소유주지분,지배기업소유주지분,지배기업소유주지분,지배기업소유주지분,지배기업소유주지분,비지배지분,자본총계,None
6,"[D210000] Statement of financial position, cur...",class3,None,현금및현금성자산,단기금융상품,기타금융자산,매출채권,기타채권,재고자산,기타자산,...,None,None,자본금,자본잉여금,기타자본항목,기타포괄손익누계액,이익잉여금,None,None,None
7,20211231,"(연결재무제표,)","88,565,366,000,000.0","12,795,554,000,000.0","6,949,333,000,000.0","12,396,646,000,000.0","3,147,296,000,000.0","4,220,970,000,000.0","11,645,641,000,000.0","2,081,853,000,000.0",...,"151,330,626,000,000.0","74,986,122,000,000.0","1,488,993,000,000.0","4,070,260,000,000.0","-1,968,385,000,000.0","-1,772,601,000,000.0","73,167,855,000,000.0","7,629,667,000,000.0","82,615,789,000,000.0","233,946,415,000,000.0"
8,20201231,"(연결재무제표,)","83,686,357,000,000.0","9,862,136,000,000.0","7,219,695,000,000.0","12,897,108,000,000.0","3,283,972,000,000.0","4,016,972,000,000.0","11,333,734,000,000.0","2,033,371,000,000.0",...,"133,003,242,000,000.0","69,480,633,000,000.0","1,488,993,000,000.0","4,190,093,000,000.0","-1,700,592,000,000.0","-3,409,661,000,000.0","68,911,800,000,000.0","6,860,337,000,000.0","76,340,970,000,000.0","209,344,212,000,000.0"
9,20191231,"(연결재무제표,)","76,082,873,000,000.0","8,681,971,000,000.0","7,292,626,000,000.0","9,449,913,000,000.0","3,513,090,000,000.0","3,402,059,000,000.0","11,663,848,000,000.0","1,777,627,000,000.0",...,"118,146,466,000,000.0","70,065,802,000,000.0","1,488,993,000,000.0","4,197,015,000,000.0","-1,516,817,000,000.0","-2,353,022,000,000.0","68,249,633,000,000.0","6,299,952,000,000.0","76,365,754,000,000.0","194,512,220,000,000.0"


현대차 재무상태표 가공

In [29]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
hy_bs_df = fs.show('bs', show_class = False, show_concept = False).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
hy_bs_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
# 영어이름이 혹시 필요하다면, 바로 아래 코드만 주석처리 해주면 된다. 지금은 일단 제거
hy_bs_df.drop(1, inplace = True)

In [30]:
hy_bs_df.columns = hy_bs_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
hy_bs_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.

In [31]:
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
hy_bs_df.rename(columns={hy_bs_df.columns[0]:'Account_year'}, inplace = True) 
hy_bs_df

,Account_year,유동자산,현금및현금성자산,단기금융상품,기타금융자산,매출채권,기타채권,재고자산,기타자산,당기법인세자산,...,부채총계,지배기업소유주지분,자본금,자본잉여금,기타자본항목,기타포괄손익누계액,이익잉여금,비지배지분,자본총계,부채와자본총계
2,20211231,"88,565,366,000,000.0","12,795,554,000,000.0","6,949,333,000,000.0","12,396,646,000,000.0","3,147,296,000,000.0","4,220,970,000,000.0","11,645,641,000,000.0","2,081,853,000,000.0","47,346,000,000.0",...,"151,330,626,000,000.0","74,986,122,000,000.0","1,488,993,000,000.0","4,070,260,000,000.0","-1,968,385,000,000.0","-1,772,601,000,000.0","73,167,855,000,000.0","7,629,667,000,000.0","82,615,789,000,000.0","233,946,415,000,000.0"
3,20201231,"83,686,357,000,000.0","9,862,136,000,000.0","7,219,695,000,000.0","12,897,108,000,000.0","3,283,972,000,000.0","4,016,972,000,000.0","11,333,734,000,000.0","2,033,371,000,000.0","138,848,000,000.0",...,"133,003,242,000,000.0","69,480,633,000,000.0","1,488,993,000,000.0","4,190,093,000,000.0","-1,700,592,000,000.0","-3,409,661,000,000.0","68,911,800,000,000.0","6,860,337,000,000.0","76,340,970,000,000.0","209,344,212,000,000.0"
4,20191231,"76,082,873,000,000.0","8,681,971,000,000.0","7,292,626,000,000.0","9,449,913,000,000.0","3,513,090,000,000.0","3,402,059,000,000.0","11,663,848,000,000.0","1,777,627,000,000.0","112,046,000,000.0",...,"118,146,466,000,000.0","70,065,802,000,000.0","1,488,993,000,000.0","4,197,015,000,000.0","-1,516,817,000,000.0","-2,353,022,000,000.0","68,249,633,000,000.0","6,299,952,000,000.0","76,365,754,000,000.0","194,512,220,000,000.0"
5,20181231,"73,008,101,000,000.0","9,113,625,000,000.0","7,936,319,000,000.0","9,755,725,000,000.0","3,595,993,000,000.0","3,291,847,000,000.0","10,714,858,000,000.0","1,770,682,000,000.0","97,271,000,000.0",...,"106,759,742,000,000.0","67,973,969,000,000.0","1,488,993,000,000.0","4,201,214,000,000.0","-1,155,244,000,000.0","-3,052,198,000,000.0","66,490,082,000,000.0","5,922,041,000,000.0","73,896,010,000,000.0","180,655,752,000,000.0"
6,20171231,"73,975,914,000,000.0","8,821,529,000,000.0","7,745,829,000,000.0","12,886,769,000,000.0","3,838,043,000,000.0","3,007,869,000,000.0","10,279,904,000,000.0","1,739,452,000,000.0","91,263,000,000.0",...,"103,442,100,000,000.0","69,103,484,000,000.0","1,488,993,000,000.0","4,201,214,000,000.0","-1,640,096,000,000.0","-2,278,955,000,000.0","67,332,328,000,000.0","5,653,870,000,000.0","74,757,354,000,000.0","178,199,454,000,000.0"
7,20161231,"72,449,600,000,000.0","7,890,089,000,000.0","7,361,735,000,000.0","12,723,993,000,000.0","4,437,552,000,000.0","3,181,030,000,000.0","10,523,812,000,000.0","1,389,803,000,000.0","46,924,000,000.0",...,"106,491,350,000,000.0","67,189,658,000,000.0","1,488,993,000,000.0","4,202,597,000,000.0","-1,640,096,000,000.0","-1,223,244,000,000.0","64,361,408,000,000.0","5,154,920,000,000.0","72,344,578,000,000.0","178,835,928,000,000.0"
8,20141231,"65,025,684,000,000.0","7,096,513,000,000.0","4,002,506,000,000.0","14,884,434,000,000.0","3,750,092,000,000.0","3,722,109,000,000.0","7,417,239,000,000.0","1,573,695,000,000.0","32,869,000,000.0",...,"84,604,552,000,000.0","57,654,828,000,000.0","1,488,993,000,000.0","4,134,550,000,000.0","-1,273,752,000,000.0","-1,344,826,000,000.0","54,649,863,000,000.0","4,965,737,000,000.0","62,620,565,000,000.0","147,225,117,000,000.0"
9,20131231,"58,856,105,000,000.0","6,872,430,000,000.0","14,875,288,000,000.0","507,821,000,000.0","3,485,345,000,000.0","3,118,386,000,000.0","7,073,116,000,000.0","1,667,936,000,000.0","54,845,000,000.0",...,"76,838,690,000,000.0","51,931,085,000,000.0","1,488,993,000,000.0","4,130,668,000,000.0","-1,128,779,000,000.0","-834,036,000,000.0","48,274,239,000,000.0","4,651,704,000,000.0","56,582,789,000,000.0","133,421,479,000,000.0"
10,20121231,"54,847,586,000,000.0","6,759,338,000,000.0","12,384,057,000,000.0","109,299,000,000.0","3,686,824,000,000.0","2,304,410,000,000.0","6,772,864,000,000.0","1,905,445,000,000.0","34,575,000,000.0",...,"73,620,239,000,000.0","44,039,059,000,000.0","1,488,993,000,000.0","4,158,988,000,000.0","-1,128,779,000,000.0","-473,

현대차 손익계산서 가공

In [32]:
hy_is_df = fs.show('is', show_class = False, show_concept = False).T.reset_index()

In [33]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
hy_is_df = fs.show('is', show_class = False, show_concept = False).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
hy_is_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
# 영어이름이 혹시 필요하다면, 바로 아래 코드만 주석처리 해주면 된다. 지금은 일단 제거
hy_is_df.drop(1, inplace = True)

In [34]:
hy_is_df.columns = hy_is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
hy_is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.

In [35]:
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
hy_is_df.rename(columns={hy_is_df.columns[0]:'Account_year'}, inplace = True) 
hy_is_df

,Account_year,매출액,매출원가,매출총이익,판매비와관리비,영업이익,공동기업및관계기업투자손익,금융수익,금융비용,기타수익,...,법인세비용,연결당기순이익,지배기업소유주지분,비지배지분,기본주당이익,보통주 기본주당이익,1우선주 기본주당이익,희석주당이익,보통주 희석주당이익,1우선주 희석주당이익
2,20210101-20211231,"117,610,626,000,000.0","95,680,131,000,000.0","21,930,495,000,000.0","15,251,546,000,000.0","6,678,949,000,000.0","1,303,365,000,000.0","912,802,000,000.0","548,410,000,000.0","1,447,261,000,000.0",...,"2,266,485,000,000.0","5,693,077,000,000.0","4,942,356,000,000.0","750,721,000,000.0",NaN,"18,979.0","19,002.0",NaN,"18,979.0","19,002.0"
3,20200101-20201231,"103,997,601,000,000.0","85,515,931,000,000.0","18,481,670,000,000.0","16,086,999,000,000.0","2,394,671,000,000.0","162,162,000,000.0","813,916,000,000.0","955,991,000,000.0","1,308,642,000,000.0",...,"168,703,000,000.0","1,924,553,000,000.0","1,424,436,000,000.0","500,117,000,000.0",NaN,"5,454.0","5,502.0",NaN,"5,454.0","5,502.0"
4,20190101-20191231,"105,746,422,000,000.0","88,091,409,000,000.0","17,655,013,000,000.0","14,049,508,000,000.0","3,605,505,000,000.0","542,826,000,000.0","827,120,000,000.0","475,218,000,000.0","1,120,958,000,000.0",...,"978,120,000,000.0","3,185,646,000,000.0","2,980,049,000,000.0","205,597,000,000.0",NaN,"11,310.0","11,355.0",NaN,"11,310.0","11,355.0"
5,20180101-20181231,"96,812,609,000,000.0","81,670,479,000,000.0","15,142,130,000,000.0","12,719,965,000,000.0","2,422,165,000,000.0","404,541,000,000.0","823,499,000,000.0","600,867,000,000.0","967,281,000,000.0",...,"884,563,000,000.0","1,645,019,000,000.0","1,508,084,000,000.0","136,935,000,000.0",NaN,"5,632.0","5,681.0",NaN,"5,632.0","5,681.0"
6,20170101-20171231,"96,376,079,000,000.0","78,798,172,000,000.0","17,577,907,000,000.0","13,003,240,000,000.0","4,574,667,000,000.0","225,053,000,000.0","972,943,000,000.0","1,120,386,000,000.0","1,153,744,000,000.0",...,"-107,850,000,000.0","4,546,400,000,000.0","4,032,824,000,000.0","513,576,000,000.0",NaN,"14,993.0","15,043.0",NaN,"14,993.0","15,043.0"
7,20160101-20161231,"93,649,024,000,000.0","75,959,720,000,000.0","17,689,304,000,000.0","12,495,804,000,000.0","5,193,500,000,000.0","1,729,447,000,000.0","1,111,238,000,000.0","678,037,000,000.0","1,177,887,000,000.0",...,"1,587,419,000,000.0","5,719,653,000,000.0","5,406,435,000,000.0","313,218,000,000.0",NaN,"20,118.0","20,156.0",NaN,"20,118.0","20,156.0"
8,20140101-20141231,"89,256,319,000,000.0","70,126,276,000,000.0","19,130,043,000,000.0","11,580,057,000,000.0","7,549,986,000,000.0","2,388,658,000,000.0","881,883,000,000.0","601,019,000,000.0","1,039,865,000,000.0",...,"2,301,806,000,000.0","7,649,468,000,000.0","7,346,807,000,000.0","302,661,000,000.0","27,037.0",NaN,NaN,"27,037.0",NaN,NaN
9,20130101-20131231,"87,307,636,000,000.0","67,859,491,000,000.0","19,448,145,000,000.0","11,132,648,000,000.0","8,315,497,000,000.0","3,057,109,000,000.0","805,261,000,000.0","552,709,000,000.0","1,138,001,000,000.0",...,"2,703,209,000,000.0","8,993,497,000,000.0","8,541,834,000,000.0","451,663,000,000.0","31,441.0",NaN,NaN,"31,441.0",NaN,NaN
10,20120101-20121231,"84,469,721,000,000.0","64,967,273,000,000.0","19,502,448,000,000.0","11,061,847,000,000.0","8,440,601,000,000.0","2,581,564,000,000.0","969,726,000,000.0","624,473,000,000.0","1,231,360,000,000.0",...,"2,549,310,000,000.0","9,061,132,000,000.0","8,566,568,000,000.0","494,564,000,000.0","31,532.0",NaN,NaN,"31,532.0",NaN,NaN


현대와 삼성의 손익계산서 필드명을 살펴 보자

In [36]:
is_df.columns

Index(['Account_year', '수익(매출액)', '매출원가', '매출총이익', '판매비와관리비', '영업이익', '기타수익',
       '기타비용', '지분법이익', '금융수익', '금융비용', '법인세비용차감전순이익(손실)', '법인세비용',
       '계속영업이익(손실)', '당기순이익(손실)', '지배기업의 소유주에게 귀속되는 당기순이익(손실)',
       '비지배지분에 귀속되는 당기순이익(손실)', '기본주당이익(손실)', '희석주당이익(손실)'],
      dtype='object', name=0)

In [37]:
hy_is_df.columns

Index(['Account_year', '매출액', '매출원가', '매출총이익', '판매비와관리비', '영업이익',
       '  공동기업및관계기업투자손익', '  금융수익', '  금융비용', '  기타수익', '  기타비용',
       '법인세비용차감전순이익', '법인세비용', '연결당기순이익', '지배기업소유주지분', '비지배지분', '기본주당이익',
       '보통주 기본주당이익', '1우선주 기본주당이익', '희석주당이익', '보통주 희석주당이익', '1우선주 희석주당이익'],
      dtype='object', name=0)

필드명이 같은 것들이 대부분이지만, 다른 경우도 존재한다. 예를 들어서, 현대는 '매출액','기본주당이익'과 삼성은 '수익(매출액)','기본주당이익(손실)'로 표기한다. 이런 표기의 다름을 해결하기 위해, 표준화된 컬럼 이름을 사용하자. 

In [38]:
# 삼성전자 code
corp_code = '00126380'

# 2019년 01월 01일 부터 연결재무제표 검색
# fs = samsung.extract_fs(bgn_de='20120101') 와 동일
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20190101')

is_df = fs.show('is', show_class = False, show_concept = True)

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

In [39]:
is_df = is_df.T.reset_index() 
is_df.columns = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.

In [40]:
is_df.columns

Index(['[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)',
       'concept_id', 'ifrs-full_Revenue', 'ifrs-full_CostOfSales',
       'ifrs-full_GrossProfit',
       'dart_TotalSellingGeneralAdministrativeExpenses',
       'dart_OperatingIncomeLoss', 'dart_OtherGains', 'dart_OtherLosses',
       'entity00126380_udf_IS_2017102191515184_IncomeStatementAbstract',
       'ifrs-full_FinanceIncome',
       'entity00126380_udf_IS_2017102191643789_IncomeStatementAbstract',
       'ifrs-full_ProfitLossBeforeTax',
       'ifrs-full_IncomeTaxExpenseContinuingOperations',
       'ifrs-full_ProfitLossFromContinuingOperations', 'ifrs-full_ProfitLoss',
       'ifrs-full_ProfitLossAttributableToOwnersOfParent',
       'ifrs-full_ProfitLossAttributableToNoncontrollingInterests',
       'ifrs-full_BasicEarningsLossPerShare',
       'ifrs-full_DilutedEarningsLossPerShare'],
      dtype='object', name=0)

In [41]:
is_df[['entity00126380_udf_IS_2017102191515184_IncomeStatementAbstract','entity00126380_udf_IS_2017102191643789_IncomeStatementAbstract']]

,entity00126380_udf_IS_2017102191515184_IncomeStatementAbstract,entity00126380_udf_IS_2017102191643789_IncomeStatementAbstract
1,지분법이익,금융비용
2,Share of net profit of associates and joint ve...,Financial expense
3,"729,614,000,000.0","7,704,554,000,000.0"
4,"506,530,000,000.0","11,318,055,000,000.0"
5,"412,960,000,000.0","8,274,871,000,000.0"
6,"539,845,000,000.0","8,608,896,000,000.0"
7,"201,442,000,000.0","8,978,913,000,000.0"
8,"19,501,000,000.0","10,706,613,000,000.0"


In [42]:
# 현대자동차 code
corp_code = '00164742'

# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()

# 현대차
hyundai = corp_list.find_by_corp_code(corp_code=corp_code)

# 2019년 01월 01일 부터 연결재무제표 검색
# fs = samsung.extract_fs(bgn_de='20120101') 와 동일
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20190101')
hy_is_df = fs.show('is', show_class = False, show_concept = True)

Annual reports:   0%|          | 0/5 [00:00<?, ?report/s]

In [43]:
hy_is_df = hy_is_df.T.reset_index() 
hy_is_df.columns = hy_is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
hy_is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.

In [44]:
hy_cols = list(hy_is_df.columns)
sam_cols = list(is_df.columns)

In [45]:
# 공통부분 찾기 

print('공통부분')
print(set(hy_cols)&set(sam_cols))
print('다른부분1')
print(set(hy_cols)-set(sam_cols))
print('다른부분2')
print(set(sam_cols)-set(hy_cols))


공통부분
{'ifrs-full_ProfitLossBeforeTax', 'ifrs-full_Revenue', 'dart_TotalSellingGeneralAdministrativeExpenses', 'dart_OperatingIncomeLoss', 'ifrs-full_GrossProfit', 'ifrs-full_IncomeTaxExpenseContinuingOperations', 'ifrs-full_CostOfSales', 'concept_id', 'ifrs-full_DilutedEarningsLossPerShare', 'ifrs-full_ProfitLossAttributableToOwnersOfParent', 'ifrs-full_FinanceIncome', 'dart_OtherLosses', 'ifrs-full_BasicEarningsLossPerShare', 'ifrs-full_ProfitLossAttributableToNoncontrollingInterests', '[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)', 'dart_OtherGains'}
다른부분1
{'entity00164742_udf_IS_20171121623772_DilutedEarningsLossPerShare', 'entity00164742_udf_IS_201811515595499_IncomeStatementAbstract', 'entity00164742_udf_IS_201711292448594_IncomeStatementAbstract', 'ifrs-full_FinanceCosts', 'entity00164742_udf_IS_201711216052534_BasicEarningsLossPerShare', 'entity00164742_udf_IS_20171121628395_DilutedEarningsLossPerShare', 'entity00164742_udf_I

In [46]:
# 현대차 손익계산표에서 다른 부분 
hy_is_df[list(set(hy_cols)-set(sam_cols))]

,entity00164742_udf_IS_20171121623772_DilutedEarningsLossPerShare,entity00164742_udf_IS_201811515595499_IncomeStatementAbstract,entity00164742_udf_IS_201711292448594_IncomeStatementAbstract,ifrs-full_FinanceCosts,entity00164742_udf_IS_201711216052534_BasicEarningsLossPerShare,entity00164742_udf_IS_20171121628395_DilutedEarningsLossPerShare,entity00164742_udf_IS_201711216150750_BasicEarningsLossPerShare
1,보통주 희석주당이익,연결당기순이익,공동기업및관계기업투자손익,금융비용,보통주 기본주당이익,1우선주 희석주당이익,1우선주 기본주당이익
2,Diluted earnings (loss) per common stock,Net income,Share of profit (loss) of associates and joint...,Finance costs,Basic earnings (loss) per common stock,Diluted earnings (loss) per preferred stock,Basic earnings (loss) per 1st preferred stock
3,"18,979.0","5,693,077,000,000.0","1,303,365,000,000.0","548,410,000,000.0","18,979.0","19,002.0","19,002.0"
4,"5,454.0","1,924,553,000,000.0","162,162,000,000.0","955,991,000,000.0","5,454.0","5,502.0","5,502.0"
5,"11,310.0","3,185,646,000,000.0","542,826,000,000.0","475,218,000,000.0","11,310.0","11,355.0","11,355.0"
6,"5,632.0","1,645,019,000,000.0","404,541,000,000.0","600,867,000,000.0","5,632.0","5,681.0","5,681.0"
7,"14,993.0","4,546,400,000,000.0","225,053,000,000.0","1,120,386,000,000.0","14,993.0","15,043.0","15,043.0"
8,"20,118.0","5,719,653,000,000.0","1,729,447,000,000.0","678,037,000,000.0","20,118.0","20,156.0","20,156.0"
9,NaN,"7,649,468,000,000.0","2,388,658,000,000.0","601,019,000,000.0",NaN,NaN,NaN
10,NaN,"8,993,497,000,000.0","3,057,109,000,000.0","552,709,000,000.0",NaN,NaN,NaN


In [27]:
hy_is_df

NameError: name 'hy_is_df' is not defined

In [47]:
# 삼성 손익계산표에서 다른 부분 
is_df[list(set(sam_cols)-set(hy_cols))]

,ifrs-full_ProfitLoss,entity00126380_udf_IS_2017102191515184_IncomeStatementAbstract,ifrs-full_ProfitLossFromContinuingOperations,entity00126380_udf_IS_2017102191643789_IncomeStatementAbstract
1,당기순이익(손실),지분법이익,계속영업이익(손실),금융비용
2,Profit (loss) for the period,Share of net profit of associates and joint ve...,Profit (loss) from continuing operations,Financial expense
3,"39,907,450,000,000.0","729,614,000,000.0","39,907,450,000,000.0","7,704,554,000,000.0"
4,"26,407,832,000,000.0","506,530,000,000.0","26,407,832,000,000.0","11,318,055,000,000.0"
5,"21,738,865,000,000.0","412,960,000,000.0","21,738,865,000,000.0","8,274,871,000,000.0"
6,"44,344,857,000,000.0","539,845,000,000.0","44,344,857,000,000.0","8,608,896,000,000.0"
7,"42,186,747,000,000.0","201,442,000,000.0","42,186,747,000,000.0","8,978,913,000,000.0"
8,"22,726,092,000,000.0","19,501,000,000.0","22,726,092,000,000.0","10,706,613,000,000.0"


In [48]:
is_df['ifrs-full_ProfitLossAttributableToNoncontrollingInterests']

1        비지배지분에 귀속되는 당기순이익(손실)
2    Non-controlling interests
3            663,659,000,000.0
4            316,986,000,000.0
5            233,811,000,000.0
6            453,980,000,000.0
7            842,178,000,000.0
8            310,437,000,000.0
Name: ifrs-full_ProfitLossAttributableToNoncontrollingInterests, dtype: object

In [49]:
hy_is_df[['entity00164742_udf_IS_201711292448594_IncomeStatementAbstract','entity00164742_udf_IS_201811515595499_IncomeStatementAbstract','entity00164742_udf_IS_201711216052534_BasicEarningsLossPerShare','entity00164742_udf_IS_201711216150750_BasicEarningsLossPerShare','entity00164742_udf_IS_20171121623772_DilutedEarningsLossPerShare','entity00164742_udf_IS_20171121628395_DilutedEarningsLossPerShare']]

,entity00164742_udf_IS_201711292448594_IncomeStatementAbstract,entity00164742_udf_IS_201811515595499_IncomeStatementAbstract,entity00164742_udf_IS_201711216052534_BasicEarningsLossPerShare,entity00164742_udf_IS_201711216150750_BasicEarningsLossPerShare,entity00164742_udf_IS_20171121623772_DilutedEarningsLossPerShare,entity00164742_udf_IS_20171121628395_DilutedEarningsLossPerShare
1,공동기업및관계기업투자손익,연결당기순이익,보통주 기본주당이익,1우선주 기본주당이익,보통주 희석주당이익,1우선주 희석주당이익
2,Share of profit (loss) of associates and joint...,Net income,Basic earnings (loss) per common stock,Basic earnings (loss) per 1st preferred stock,Diluted earnings (loss) per common stock,Diluted earnings (loss) per preferred stock
3,"1,303,365,000,000.0","5,693,077,000,000.0","18,979.0","19,002.0","18,979.0","19,002.0"
4,"162,162,000,000.0","1,924,553,000,000.0","5,454.0","5,502.0","5,454.0","5,502.0"
5,"542,826,000,000.0","3,185,646,000,000.0","11,310.0","11,355.0","11,310.0","11,355.0"
6,"404,541,000,000.0","1,645,019,000,000.0","5,632.0","5,681.0","5,632.0","5,681.0"
7,"225,053,000,000.0","4,546,400,000,000.0","14,993.0","15,043.0","14,993.0","15,043.0"
8,"1,729,447,000,000.0","5,719,653,000,000.0","20,118.0","20,156.0","20,118.0","20,156.0"
9,"2,388,658,000,000.0","7,649,468,000,000.0",NaN,NaN,NaN,NaN
10,"3,057,109,000,000.0","8,993,497,000,000.0",NaN,NaN,NaN,NaN


In [50]:
is_df.T.reset_index()

,0,1,2,3,4,5,6,7,8
0,"[D310000] Income statement, by function of exp...","[D310000] Income statement, by function of exp...","[D310000] Income statement, by function of exp...",20210101-20211231,20200101-20201231,20190101-20191231,20180101-20181231,20170101-20171231,20160101-20161231
1,concept_id,label_ko,label_en,"(연결재무제표,)","(연결재무제표,)","(연결재무제표,)","(연결재무제표,)","(연결재무제표,)","(연결재무제표,)"
2,ifrs-full_Revenue,수익(매출액),Revenue(Sales),"279,604,799,000,000.0","236,806,988,000,000.0","230,400,881,000,000.0","243,771,415,000,000.0","239,575,376,000,000.0","201,866,745,000,000.0"
3,ifrs-full_CostOfSales,매출원가,Cost of sales,"166,411,342,000,000.0","144,488,296,000,000.0","147,239,549,000,000.0","132,394,411,000,000.0","129,290,661,000,000.0","120,277,715,000,000.0"
4,ifrs-full_GrossProfit,매출총이익,Gross profit,"113,193,457,000,000.0","92,318,692,000,000.0","83,161,332,000,000.0","111,377,004,000,000.0","110,284,715,000,000.0","81,589,030,000,000.0"
5,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,Selling and administrative expenses,"61,559,601,000,000.0","56,324,816,000,000.0","55,392,823,000,000.0","52,490,335,000,000.0","56,639,677,000,000.0","52,348,358,000,000.0"
6,dart_OperatingIncomeLoss,영업이익,Operating profit,"51,633,856,000,000.0","35,993,876,000,000.0","27,768,509,000,000.0","58,886,669,000,000.0","53,645,038,000,000.0","29,240,672,000,000.0"
7,dart_OtherGains,기타수익,Other non-operating income,"2,205,695,000,000.0","1,384,068,000,000.0","1,778,666,000,000.0","1,485,037,000,000.0","3,010,657,000,000.0","3,238,261,000,000.0"
8,dart_OtherLosses,기타비용,Other non-operating expense,"2,055,971,000,000.0","2,488,902,000,000.0","1,414,707,000,000.0","1,142,018,000,000.0","1,419,648,000,000.0","2,463,814,000,000.0"
9,entity00126380_udf_IS_2017102191515184_IncomeS...,지분법이익,Share of net profit of associates and joint ve...,"729,614,000,000.0","506,530,000,000.0","412,960,000,000.0","539,845,000,000.0","201,442,000,000.0","19,501,000,000.0"


In [51]:
hy_is_df

,"[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)",concept_id,ifrs-full_Revenue,ifrs-full_CostOfSales,ifrs-full_GrossProfit,dart_TotalSellingGeneralAdministrativeExpenses,dart_OperatingIncomeLoss,entity00164742_udf_IS_201711292448594_IncomeStatementAbstract,ifrs-full_FinanceIncome,ifrs-full_FinanceCosts,...,ifrs-full_IncomeTaxExpenseContinuingOperations,entity00164742_udf_IS_201811515595499_IncomeStatementAbstract,ifrs-full_ProfitLossAttributableToOwnersOfParent,ifrs-full_ProfitLossAttributableToNoncontrollingInterests,ifrs-full_BasicEarningsLossPerShare,entity00164742_udf_IS_201711216052534_BasicEarningsLossPerShare,entity00164742_udf_IS_201711216150750_BasicEarningsLossPerShare,ifrs-full_DilutedEarningsLossPerShare,entity00164742_udf_IS_20171121623772_DilutedEarningsLossPerShare,entity00164742_udf_IS_20171121628395_DilutedEarningsLossPerShare
1,"[D310000] Income statement, by function of exp...",label_ko,매출액,매출원가,매출총이익,판매비와관리비,영업이익,공동기업및관계기업투자손익,금융수익,금융비용,...,법인세비용,연결당기순이익,지배기업소유주지분,비지배지분,기본주당이익,보통주 기본주당이익,1우선주 기본주당이익,희석주당이익,보통주 희석주당이익,1우선주 희석주당이익
2,"[D310000] Income statement, by function of exp...",label_en,Sales,Cost of sales,Gross profit,Selling and administrative expenses,Operating income(loss),Share of profit (loss) of associates and joint...,Finance income,Finance costs,...,Income tax expense (benefit),Net income,"Profit (loss), attributable to owners of parent","Profit (loss), attributable to non-controlling...",Basic earnings (loss) per share,Basic earnings (loss) per common stock,Basic earnings (loss) per 1st preferred stock,Diluted earnings (loss) per share,Diluted earnings (loss) per common stock,Diluted earnings (loss) per preferred stock
3,20210101-20211231,"(연결재무제표,)","117,610,626,000,000.0","95,680,131,000,000.0","21,930,495,000,000.0","15,251,546,000,000.0","6,678,949,000,000.0","1,303,365,000,000.0","912,802,000,000.0","548,410,000,000.0",...,"2,266,485,000,000.0","5,693,077,000,000.0","4,942,356,000,000.0","750,721,000,000.0",NaN,"18,979.0","19,002.0",NaN,"18,979.0","19,002.0"
4,20200101-20201231,"(연결재무제표,)","103,997,601,000,000.0","85,515,931,000,000.0","18,481,670,000,000.0","16,086,999,000,000.0","2,394,671,000,000.0","162,162,000,000.0","813,916,000,000.0","955,991,000,000.0",...,"168,703,000,000.0","1,924,553,000,000.0","1,424,436,000,000.0","500,117,000,000.0",NaN,"5,454.0","5,502.0",NaN,"5,454.0","5,502.0"
5,20190101-20191231,"(연결재무제표,)","105,746,422,000,000.0","88,091,409,000,000.0","17,655,013,000,000.0","14,049,508,000,000.0","3,605,505,000,000.0","542,826,000,000.0","827,120,000,000.0","475,218,000,000.0",...,"978,120,000,000.0","3,185,646,000,000.0","2,980,049,000,000.0","205,597,000,000.0",NaN,"11,310.0","11,355.0",NaN,"11,310.0","11,355.0"
6,20180101-20181231,"(연결재무제표,)","96,812,609,000,000.0","81,670,479,000,000.0","15,142,130,000,000.0","12,719,965,000,000.0","2,422,165,000,000.0","404,541,000,000.0","823,499,000,000.0","600,867,000,000.0",...,"884,563,000,000.0","1,645,019,000,000.0","1,508,084,000,000.0","136,935,000,000.0",NaN,"5,632.0","5,681.0",NaN,"5,632.0","5,681.0"
7,20170101-20171231,"(연결재무제표,)","96,376,079,000,000.0","78,798,172,000,000.0","17,577,907,000,000.0","13,003,240,000,000.0","4,574,667,000,000.0","225,053,000,000.0","972,943,000,000.0","1,120,386,000,000.0",...,"-107,850,000,000.0","4,546,400,000,000.0","4,032,824,000,000.0","513,576,000,000.0",NaN,"14,993.0","15,043.0",NaN,"14,993.0","15,043.0"
8,20160101-20161231,"(연결재무제표,)","93,649,024,000,000.0","75,959,720,000,000.0","17,689,304,000,000.0","12,495,804,000,000.0","5,193,500,000,000.0","1,729,447,000,000.0","1,111,238,000,000.0","678,037,000,000.0",...,"1,587,419,000,000.0","5,719,653,000,000.0","5,406,435,000,000.0","313,218,000,000.0",NaN,"20,118.0","20,156.0",NaN,"20,118.0","20,156.0"
9,20140101-20141231,"(연결재무제표,)","89,256,319,000,000.0","70,126,276,000,000.0","19,130,043,000,000.0","11,580,057,000,000.0","7,549,986,000,000.0","2,388,658,000,000.0","881,883,000,00

# Solution
xbrl을 이용해보자

In [62]:
dart.api.finance.xbrl_taxonomy(sj_div='BS1')

{'status': '000',
 'message': '정상',
 'list': [{'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_StatementOfFinancialPositionAbstract',
   'account_nm': 'StatementOfFinancialPositionAbstract',
   'label_kor': '재무상태표 [abstract]',
   'label_eng': 'Statement of financial position [abstract]',
   'ifrs_ref': ' '},
  {'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_AssetsAbstract',
   'account_nm': 'AssetsAbstract',
   'label_kor': '자산 [abstract]',
   'label_eng': 'Assets [abstract]',
   'ifrs_ref': ' '},
  {'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_CurrentAssets',
   'account_nm': 'CurrentAssets',
   'label_kor': '유동자산',
   'label_eng': 'Current assets',
   'data_tp': 'X',
   'ifrs_ref': 'K-IFRS 1001 문단 60'},
  {'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_CashAndCashEquivalents',
   'account_nm': 'CashAndCashEquivalents',
   'label_kor': '현금및현금성자산',
   'label_eng': 'Cash and cash equivalents',
   'data_tp': 'X'

### To dos 
- 유지보수를 위한 tutorial 노트북을 만들자. 
- 깔끔하게 위의 과정들을 Class나 함수로 구현하는 노트북을 하나 더 만들자.  
- 테이블 정의서에 맞도록 bf


In [52]:
import dart_fss as dart


# 삼성전자 code
corp_code = '00126380'

# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()

# 삼성전자
samsung = corp_list.find_by_corp_code(corp_code=corp_code)



In [58]:
xbrl

,Document information [text block]
Document title,Annual Report
Amendment,false
Document creation date,2021-12-31
Restatement of comparative financial statements,false
Company name,"Samsung Electronics Co., Ltd."


In [55]:
# 사업보고서 검색
reports = samsung.search_filings(bgn_de='20190101', pblntf_detail_ty='a001')

# 첫번째 리포트 선택
report = reports[0]

# 리포트의 xbrl 데이터
xbrl = report.xbrl

# 연결재무제표 존재 여부 확인( True / False)
xbrl.exist_consolidated()

# 감사 정보 (영문) -> DataFrame 형태로 반환됨
audit = xbrl.get_audit_information(lang='en')

# 연결 현금흐름표 추출 (리스트 반환)
cf = xbrl.get_cash_flows()

# 연결 현금프름표
cf = cf[0]

# Pandas DataFrame으로 변환
df = cf.to_DataFrame()

# Pandas DataFrame으로 변환 / 분류 정보 제외
df_wo_class = cf.to_DataFrame(show_class=True)
df_wo_class

Output()

Output()

[D520000] Statement of cash flows, indirect method - Consolidated financial statements (Unit: KRW)  \
                                                                                           concept_id   
0    ifrs-full_CashFlowsFromUsedInOperatingActivities                                                   
1   entity00126380_udf_CF_201710211099539_CashFlow...                                                   
2   entity00126380_udf_CF_201710211092510_udf_CF_2...                                                   
3   entity00126380_udf_CF_2017102110929616_udf_CF_...                                                   
4   entity00126380_udf_CF_2017102110934928_udf_CF_...                                                   
5   ifrs-full_InterestReceivedClassifiedAsOperatin...                                                   
6   ifrs-full_InterestPaidClassifiedAsOperatingAct...                                                   
7   ifrs-full_DividendsReceivedClassifiedAsOperati...                                                   
8   ifrs-full_IncomeTaxesPaidRefundClassifiedAsOpe...                                                   
9    ifrs-full_CashFlowsFromUsedInInvestingActivities                                                   
10  entity00126380_udf_CF_2017102110167395_CashFlo...                                                   
11  entity00126380_udf_CF_201987152650384_CashFlow...                                                   
12  entity00126380_udf_CF_2019726153251267_CashFlo...                                                   
13  dart_ProceedsFromSalesOfLongTermFinancialInstr...                                                   
14        dart_PurchaseOfLongTermFinancialInstruments                                                   
15  entity00126380_udf_CF_202182133547112_CashFlow...                                                   
16  entity00126380_udf_CF_2022221103235427_CashFlo...                                                   
17  entity00126380_udf_CF_2018510204038291_CashFlo...                                                   
18  entity00126380_udf_CF_2018510204040276_CashFlo...                                                   
19  entity00126380_udf_CF_2018510204036276_CashFlo...                                                   
20  entity00126380_udf_CF_2018510204025664_CashFlo...                                                   
21  entity00126380_udf_CF_202182133252272_CashFlow...                                                   
22  entity00126380_udf_CF_20171021101629633_CashFl...                                                   
23  ifrs-full_ProceedsFromSalesOfPropertyPlantAndE...                                                   
24  ifrs-full_PurchaseOfPropertyPlantAndEquipmentC...                                                   
25  ifrs-full_ProceedsFromSalesOfIntangibleAssetsC...                                                   
26  ifrs-full_PurchaseOfIntangibleAssetsClassified...                                                   
27  entity00126380_udf_CF_20171021101626447_CashFl...                                                   
28  entity00126380_udf_CF_202182134217299_CashFlow...                                                   
29  entity00126380_udf_CF_20171021101625672_CashFl...                                                   
30   ifrs-full_CashFlowsFromUsedInFinancingActivities                                                   
31  entity00126380_udf_CF_20171021102249949_CashFl...                                                   
32  entity00126380_udf_CF_2021111023423146_CashFlo...                                                   
33                dart_RepaymentsOfLongTermBorrowings                                                   
34  ifrs-full_DividendsPaidClassifiedAsFinancingAc...                                                   
35  entity00126380_udf_CF_20171021102253395_CashFl...                                                   
36  entity001

In [ ]:
entity00126380_udf_CF_201710211099539_CashFlow...